In [0]:
from pyspark.sql.functions import col, round
from pyspark.sql.types import IntegerType

In [0]:

configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "{APP_CLIENT_ID}",
    "fs.azure.account.oauth2.client.secret": '{APP_SECRECT_KEY}',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{APP_TENANT_ID}/oauth2/token"
}

In [0]:

if any(mount.mountPoint == '/mnt/olympicdatamount' for mount in dbutils.fs.mounts()):    
    dbutils.fs.unmount('/mnt/olympicdatamount')
dbutils.fs.mount(

    source = "abfss://tokyo-olympic-data-container@adityaolympicstorage.dfs.core.windows.net/",
    mount_point = "/mnt/olympicdatamount",
    extra_configs = configs
)


/mnt/olympicdatamount has been unmounted.


True

In [0]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdatamount/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdatamount/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdatamount/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdatamount/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/olympicdatamount/raw-data/teams.csv")

In [0]:
# Find the top 5 countries with the highest number of gold medals
top_countries = medals.orderBy("Gold", ascending=False).limit(5)
top_countries.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+



In [0]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn("avg_female", round(col("Female") / col("Total"), 2)) \
    .withColumn("avg_male", round(col("Male") / col("Total"), 2))
average_entries_by_gender.show()

+--------------------+------+----+-----+----------+--------+
|          Discipline|Female|Male|Total|avg_female|avg_male|
+--------------------+------+----+-----+----------+--------+
|      3x3 Basketball|    32|  32|   64|       0.5|     0.5|
|             Archery|    64|  64|  128|       0.5|     0.5|
| Artistic Gymnastics|    98|  98|  196|       0.5|     0.5|
|   Artistic Swimming|   105|   0|  105|       1.0|     0.0|
|           Athletics|   969|1072| 2041|      0.47|    0.53|
|           Badminton|    86|  87|  173|       0.5|     0.5|
|   Baseball/Softball|    90| 144|  234|      0.38|    0.62|
|          Basketball|   144| 144|  288|       0.5|     0.5|
|    Beach Volleyball|    48|  48|   96|       0.5|     0.5|
|              Boxing|   102| 187|  289|      0.35|    0.65|
|        Canoe Slalom|    41|  41|   82|       0.5|     0.5|
|        Canoe Sprint|   123| 126|  249|      0.49|    0.51|
|Cycling BMX Frees...|    10|   9|   19|      0.53|    0.47|
|  Cycling BMX Racing|  

In [0]:
# write top_countries and average_entries_by_gender to csv files
top_countries.write.mode("overwrite").option("header", True).csv("/mnt/olympicdatamount/transformed-data/top_countries")
average_entries_by_gender.write.mode("overwrite").option("header", True).csv("/mnt/olympicdatamount/transformed-data/average_entries_by_gender")